In [1]:
import pandas as pd

data_NY=pd.read_csv('New_York_reviews.csv')
data_Madrid=pd.read_csv('Madrid_reviews.csv')
data_Deli=pd.read_csv('New_Delhi_reviews.csv')

/tmp/ipython-input-1-2740933268.py:3: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_NY=pd.read_csv('New_York_reviews.csv')
/tmp/ipython-input-1-2740933268.py:5: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data_Deli=pd.read_csv('New_Delhi_reviews.csv')


In [2]:
data_NY.head()

,Unnamed: 0,parse_count,restaurant_name,rating_review,sample,review_id,title_review,review_preview,review_full,date,city,url_restaurant,author_id
0,1,2,Lido,5,Positive,review_773559838,A Regular Treat,My wife and I have been eating dinner frequent...,My wife and I have been eating dinner frequent...,"October 8, 2020",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_0
1,2,3,Lido,4,Positive,review_769429529,Good neighborhood spot!,Came with family for Labor Day weekend brunch ...,Came with family for Labor Day weekend brunch ...,"September 8, 2020",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_1
2,3,4,Lido,1,Negative,review_745700258,Disappointing,Food was mediocre at best. The lamb chops are...,Food was mediocre at best. The lamb chops are ...,"February 17, 2020",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_2
3,4,5,Lido,5,Positive,review_728859349,What a find in Harlem,My co-workers were volunteering at a foodbank ...,My co-workers were volunteering at a foodbank ...,"November 25, 2019",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_3
4,5,6,Lido,5,Positive,review_728429643,Lunch,Lido is an intimate boutique style restaurant....,Lido is an intimate boutique style restaurant....,"November 23, 2019",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_4


In [3]:
data = pd.concat([data_NY, data_Madrid, data_Deli], ignore_index=True).sort_values(by='date').reset_index()
data

,index,Unnamed: 0,parse_count,restaurant_name,rating_review,sample,review_id,title_review,review_preview,review_full,date,city,url_restaurant,author_id
0,477959,484663,484664,Frankie_Johnnie_s_Steakhouse,4,Positive,review_3323403,"A memorable experience, to say the least","When we first walked through the door, the pos...","When we first walked through the door, the pos...","April 1, 2005",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_239340
1,418038,423895,423896,Cara_Mia,4,Positive,review_4827414,Good Pre-Theater,"Everybody knows it's small and noisy, but... G...","Everybody knows it's small and noisy, but... G...","April 1, 2006",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_218477
2,167673,170033,170034,Katz_s_Deli,4,Positive,review_7271578,Very much a New York cultural experience,We visited this restaurant as part of the Marv...,We visited this restaurant as part of the Marv...,"April 1, 2007",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_113493
3,308478,312790,312791,Ellen_s_Stardust_Diner,4,Positive,review_7271514,Great stop before a Broadway show,We visited Ellen's just before going to see Je...,We visited Ellen's just before going to see Je...,"April 1, 2007",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_113493
4,256650,260171,260172,Virgil_s_Real_BBQ_Times_Square,5,Positive,review_7270199,Terrific BBQ and outstanding service!,The four of us visited Virgils for lunch on ou...,The four of us visited Virgils for lunch on ou...,"April 1, 2007",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_113493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886078,818381,80469,80470,Hi_Mircheee,3,Negative,review_217165049,over priced,this is a over priced restro with no taste in ...,NaN,NaN,NaN,NaN,NaN
886079,818382,80469,80470,Hi_Mircheee,3,Negative,review_217165049,over priced,this is a over priced restro with no taste in ...,NaN,NaN,NaN,NaN,NaN
886080,818383,most of the time u end up wating 30 40 min fo...,this is a over priced restro with no taste in ...,"July 23, 2014",New_Delhi_National_Capital_Territory_of_Delhi,https://www.tripadvisor.com/Restaurant_Review-...,UID_7681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886081,879666,142015,142016,The_Great_Kabab_Factory,5,Positive,review_201442126,awesome starters and desserts,well this review is a bit late as i dined here...,NaN,NaN,NaN,NaN,NaN


In [4]:
data_reviews = data[['review_full','sample']][(data['sample']=='Positive')|(data['sample']=='Negative')]
data_reviews

,review_full,sample
0,"When we first walked through the door, the pos...",Positive
1,"Everybody knows it's small and noisy, but... G...",Positive
2,We visited this restaurant as part of the Marv...,Positive
3,We visited Ellen's just before going to see Je...,Positive
4,The four of us visited Virgils for lunch on ou...,Positive
...,...,...
886076,NaN,Negative
886077,NaN,Negative
886078,NaN,Negative
886079,NaN,Negative


In [5]:
import numpy as np

conditions = [
    (data_reviews['sample'] == 'Positive'),
    (data_reviews['sample'] == 'Negative'),
]

choices = [1, 0]

data_reviews['answer'] = np.select(conditions, choices)
data_reviews=data_reviews.drop('sample', axis=1)
data_reviews

,review_full,answer
0,"When we first walked through the door, the pos...",1
1,"Everybody knows it's small and noisy, but... G...",1
2,We visited this restaurant as part of the Marv...,1
3,We visited Ellen's just before going to see Je...,1
4,The four of us visited Virgils for lunch on ou...,1
...,...,...
886076,NaN,0
886077,NaN,0
886078,NaN,0
886079,NaN,0


In [6]:
data_reviews=data_reviews.dropna()
data_reviews

,review_full,answer
0,"When we first walked through the door, the pos...",1
1,"Everybody knows it's small and noisy, but... G...",1
2,We visited this restaurant as part of the Marv...,1
3,We visited Ellen's just before going to see Je...,1
4,The four of us visited Virgils for lunch on ou...,1
...,...,...
834902,"The atmosphere of the restaurant is amazing, i...",1
834903,We had a dinner with colleagues at the terrace...,1
834904,Went to NYC for a few days to see if it was as...,1
834905,"I order from Domino's alot of times, but these...",0


In [11]:
#посмотрим, какие основные причины негативных отзывов:

from sklearn.feature_extraction.text import CountVectorizer  #превращает текст в "мешок слов"

vectorizer = CountVectorizer(stop_words='english', ngram_range=(3, 3))  # убираем предлоги и всё такое + берем не слова отдельно а фразы по 3 слова
X = vectorizer.fit_transform(data_reviews[data_reviews['answer']==0]['review_full']) #берем только негативные отзывы

# Суммируем частоты по всем отзывам
word_counts = X.sum(axis=0).A1  # переводим в одномерный массив
words = vectorizer.get_feature_names_out()
word_freq = dict(zip(words, word_counts))

top_words = pd.Series(word_freq).sort_values(ascending=False).head(10)
print(top_words)

food good service         1117
new york city              904
food just ok               813
service good food          770
really looking forward     509
food pretty good           480
food good great            468
food just average          441
took long time             434
food average service       379
dtype: int64


In [ ]:
#понятно, что большая часть негативных отзывов выглядит как "... хорошо, но....".
#по n-граммам из 3х слов из части "но..." нашли "took long time".
#то есть видна проблема долгого ожидания

In [12]:
vectorizer = CountVectorizer(stop_words='english', ngram_range=(4, 4))  # теперь фразы по 4 слова
X = vectorizer.fit_transform(data_reviews[data_reviews['answer']==0]['review_full']) #берем только негативные отзывы

# Суммируем частоты по всем отзывам
word_counts = X.sum(axis=0).A1  # переводим в одномерный массив
words = vectorizer.get_feature_names_out()
word_freq = dict(zip(words, word_counts))

top_words = pd.Series(word_freq).sort_values(ascending=False).head(10)
print(top_words)

don waste time money         128
food good service good        90
left bad taste mouth          90
good food good service        85
really wanted like place      78
service good food average     73
service good food good        72
food took long time           67
food just ok service          63
recent trip new york          61
dtype: int64


In [ ]:
#тут то же took long time + left bad taste mouth (то есть есть жалобы и на саму кухню)

In [ ]:
#это глобально . для конкретного ресторана будет смотреть интереснее

In [ ]:
#напишем алгоритм классификации отзывов на положительные и отрицательные. для начала переведем тексты в числовые признаки

In [7]:
from sklearn.model_selection import train_test_split


# разделим наш датасет (data_reviews['review_full'] и data_reviews['answer'])
# на тренировочную и тестовую выборки,
# выделив 20% датасета в качестве теста (с помощью аргумента test_size),
# передадим также наши метки классов в качестве параметра stratify
# для сбалансированного разделения на обучающую и тестовую выборки,
# зафиксируем random_state

x_train, x_test, y_train, y_test = train_test_split(
    data_reviews['review_full'], data_reviews['answer'], test_size=0.2,
    stratify=data_reviews['answer'], random_state=42)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Создадим инстанс TfidfVectorizer, используя следующие параметры:
# используем униграммы, задав значение (1,1) для аргумента ngram_range,
# текст будем делить на слова, задав значение 'word' аргумента analyzer,
# зададим значение min_df равное 2, чтобы в словарь добавлялись все ngrams, встретившиеся больше 2 раз,
# зададим стоп-слова "english" с помощью аргумента stop_words

vectorizer = TfidfVectorizer(ngram_range=(1,1),
                             analyzer='word',
                             min_df=2,
                             stop_words="english")

# обучаем словарь и сразу преобразуем обучающую выборку x_train с помощью метода fit_transform
tfidf_train = vectorizer.fit_transform(x_train)
# преобразуем тестовую выборку x_test с помощью метода transform
tfidf_test = vectorizer.transform(x_test)

In [9]:
import scipy


# попробуем преобразовать первые 1024 строк матрицы tfidf_train
# с помощью метода scipy.sparse.csr_matrix.todense
train_tfidf_dense = scipy.sparse.csr_matrix.todense(tfidf_train[:1024, :])
print(train_tfidf_dense[0]) # [train_tfidf_dense[0] != 0][0]
print(len(train_tfidf_dense))
print(train_tfidf_dense[0][train_tfidf_dense[0] != 0][0])

[[0. 0. 0. ... 0. 0. 0.]]
1024
[[0.19075715 0.17513284 0.3341779  0.22663283 0.06721133 0.12685865
  0.1585714  0.13452869 0.14424884 0.21697073 0.09695643 0.11443215
  0.2042706  0.22062824 0.2640121  0.32189083 0.37023929 0.27462413
  0.19630147 0.22020772 0.16115877 0.16265752]]


In [10]:
train_tfidf_dense.shape

(1024, 82617)

In [ ]:
#теперь нейронная сеть:

In [11]:
# установим последнюю версию wandb
!pip install -q --upgrade wandb

In [12]:
import os
import wandb # для логирования

import numpy as np
import random
from tqdm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim # для оптимизаторов
from torchvision import datasets # для данных
import torchvision.transforms as transforms # для преобразований тензоров

import matplotlib.pyplot as plt

In [13]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [14]:
# Зафиксируем seed для воспроизводимости

def seed_everything(seed):
    random.seed(seed) # фиксируем генератор случайных чисел
    os.environ['PYTHONHASHSEED'] = str(seed) # фиксируем заполнения хешей
    np.random.seed(seed) # фиксируем генератор случайных чисел numpy
    torch.manual_seed(seed) # фиксируем генератор случайных чисел pytorch
    torch.cuda.manual_seed(seed) # фиксируем генератор случайных чисел для GPU
    #torch.backends.cudnn.deterministic = True # выбираем только детерминированные алгоритмы (для сверток)
    #torch.backends.cudnn.benchmark = False # фиксируем алгоритм вычисления сверток

In [15]:
# функция перевода класса конфигурации в словарь

def class2dict(f):
  return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

In [16]:
class CFG:

# Задаем параметры нашего эксперимента

  api = "4bc1ab80ba6cc5cf38f824b647f74e0e95f3f2c6"# вписать свой API Wandb
  project = "test-project"# вписать название эксперимента, который предварительно надо создать в Wandb
  entity = "ekteam"# ввести свой логин
  num_epochs = 10 # количество эпох
  train_batch_size = 32 # размер батча обучающей выборки
  test_batch_size = 16 # размер батча тестовой выборки
  num_workers = 2 # количество активных процессов на загрузку данных
  lr = 0.001 # learning_rate
  seed = 2022 # для функции воспроизводимости
  wandb = True # флаг использования Wandb

In [17]:
X_train_tensor = torch.tensor(train_tfidf_dense, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train[:1024].values, dtype=torch.long)

test_tfidf_dense = scipy.sparse.csr_matrix.todense(tfidf_test[:256, :])
X_test_tensor = torch.tensor(test_tfidf_dense, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test[:256].values, dtype=torch.long)

In [18]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=CFG.test_batch_size, shuffle=False)

In [72]:
#+попробовуем разные архитектуры для нейронной сети

In [19]:
class Net_1(nn.Module): # наследуемся от класса nn.Module
    def __init__(self, input_dim, hidden_1=256, hidden_2=128): # количество нейронов в скрытых слоях (256, 128)
        super(Net_1,self).__init__()

        # первый слой (input_dim=82617 -> hidden_1)  # число признаков из TFIDF
        self.fc1 = nn.Linear(input_dim, hidden_1)
        # второй слой (hidden_1 -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # третий (выходной) слой (hidden_2 -> 2 класса)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self,x):
        # добавляем первый скрытый слой с ReLU активацией
        x = F.relu(self.fc1(x))
        # добавляем второй скрытый слой с ReLU активацией
        x = F.relu(self.fc2(x))
        # добавляем третий (выходной) слой
        x = self.fc3(x)
        return x

In [20]:
class Net_2(nn.Module): # наследуемся от класса nn.Module
    def __init__(self, input_dim, hidden_1=256, hidden_2=128, hidden_3 = 64):
        super(Net_2,self).__init__()
        self.net = torch.nn.Sequential(
                      torch.nn.Linear(input_dim, hidden_1),
                      torch.nn.ReLU(),
                      torch.nn.Linear(hidden_1, hidden_2),
                      torch.nn.ReLU(),
                      torch.nn.Linear(hidden_2, hidden_3),
                      torch.nn.ReLU(),
                      torch.nn.Linear(hidden_3, 2)
                    )

    def forward(self,x):
        x = self.net(x)
        return x

In [39]:
class Net_3(nn.Module):
    def __init__(self, input_dim, hidden_1=256, hidden_2=128):
        super(Net_3, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_1)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hidden_2, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [23]:
model_1 = Net_1(input_dim=82617)
model_1 = model_1.cuda() # переводим модель на GPU
print(model_1) # посмотрим на нашу модель

Net_1(
  (fc1): Linear(in_features=82617, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)


In [24]:
model_2 = Net_2(input_dim=82617)
model_2 = model_2.cuda()
print(model_2)

Net_2(
  (net): Sequential(
    (0): Linear(in_features=82617, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=2, bias=True)
  )
)


In [40]:
model_3 = Net_3(input_dim=82617)
model_3 = model_3.cuda()
print(model_3)

Net_3(
  (fc1): Linear(in_features=82617, out_features=256, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=128, out_features=2, bias=True)
)


In [41]:
# функция потерь (cross-entropy) для 2 классов
criterion = nn.CrossEntropyLoss()

optimizer_1 = torch.optim.Adam(model_1.parameters(),lr = 0.001)
optimizer_2 = torch.optim.Adam(model_2.parameters(),lr = 0.001)
optimizer_3 = torch.optim.Adam(model_3.parameters(),lr = 0.001)

In [27]:
# функция обучения модели
def train(model, device, train_loader, optimizer, criterion, epoch, WANDB):
    model.train() # обязательно переводим в режим обучения
    test_loss = 0
    correct = 0

    n_ex = len(train_loader)

    for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=n_ex):
        data, target = data.to(device), target.to(device) # переводим картинки и таргеты на GPU
        # обнуляем градиенты!
        optimizer.zero_grad()
        # прямой проход
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item() # определяем корректность предсказания
        train_loss = criterion(output, target) # считаем значение функции потерь
        # обратный проход
        train_loss.backward()
        # делаем градиентный шаг оптимизатором
        optimizer.step()
        # считаем метрики и лосс
    tqdm.write('\nTrain set: Average loss: {:.4f}, Accuracy: {:.0f}%'.format(
        train_loss, 100. * correct / len(train_loader.dataset)))

    if WANDB:
        wandb.log({'train_loss': train_loss,
                   'train_accuracy': correct / len(train_loader.dataset)})


# функция тестирования
def test(model, device, test_loader, criterion, WANDB):
    model.eval() # переводем модель в режим инференса
    test_loss = 0
    correct = 0
    all_pred=[]
    # показываем, что обученич нет и градиенты не обновляются
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss = criterion(output, target) # считаем значение функции потерь
            pred = output.argmax(dim=1, keepdim=True)
            all_pred.append(pred)
            correct += pred.eq(target.view_as(pred)).sum().item()
            # считаем метрики
    tqdm.write('Test set: Average loss: {:.4f}, Accuracy: {:.0f}%'.format(
        test_loss, 100. * correct / len(test_loader.dataset)))

    if WANDB:
        wandb.log({'test_loss': test_loss,
                   'test_accuracy': correct / len(test_loader.dataset)})
    all_pred = torch.cat(all_pred, dim=0)  # shape: [total_samples, 1]
    all_pred = all_pred.squeeze(dim=1)
    return all_pred

# основная функция для экспериментов
def main(model, optimizer):
    # настраиваем окружение для wandb
    if CFG.wandb:
        os.environ["WANDB_API_KEY"] = CFG.api
        wandb.init(project=CFG.project, entity=CFG.entity, reinit=True, config=class2dict(CFG))

    use_cuda = torch.cuda.is_available() # проверяем доступность GPU

    seed_everything(CFG.seed) # фиксируем сиды

    device = torch.device("cuda" if use_cuda else "cpu") # выделили устройство

    if CFG.wandb:
        wandb.watch(model, log='all') # логируем все (метрики, лоссы, градиенты)

    for epoch in range(1, CFG.num_epochs + 1): # цикл на эпохи
        print('\nEpoch:', epoch)
        train(model, device, train_loader, optimizer, criterion, epoch, CFG.wandb)
        pred=test(model, device, test_loader, criterion, CFG.wandb)
    print('Training is ended!')
    return pred

In [28]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Sun Jun 22 10:24:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P0             27W /   70W |     350MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [29]:
CFG.wandb

True

In [30]:
res_1=main(model_1, optimizer_1)

wandb: Currently logged in as: mogilnikoff2 (ekteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.



Epoch: 1


100%|██████████| 32/32 [00:00<00:00, 34.86it/s]



Train set: Average loss: 0.3644, Accuracy: 82%
Test set: Average loss: 0.5685, Accuracy: 80%

Epoch: 2


100%|██████████| 32/32 [00:00<00:00, 95.46it/s]


...


Train set: Average loss: 0.0002, Accuracy: 100%
Test set: Average loss: 1.1521, Accuracy: 88%

Epoch: 9


100%|██████████| 32/32 [00:00<00:00, 89.49it/s]



Train set: Average loss: 0.0004, Accuracy: 100%
Test set: Average loss: 1.1788, Accuracy: 88%

Epoch: 10


100%|██████████| 32/32 [00:00<00:00, 92.10it/s]


Train set: Average loss: 0.0001, Accuracy: 100%
Test set: Average loss: 1.2081, Accuracy: 88%
Training is ended!


In [31]:
res_2=main(model_2, optimizer_2)

test_accuracy,▁▁▄▅█▇▇▇▇▇
test_loss,▁▁▃▆▆▇▇▇██
train_accuracy,▁▁▃███████
train_loss,█▄▃▁▁▁▁▁▁▁
test_accuracy,0.87891
test_loss,1.20809
train_accuracy,1
train_loss,0.00011



Epoch: 1


100%|██████████| 32/32 [00:00<00:00, 90.70it/s]



Train set: Average loss: 0.3654, Accuracy: 79%
Test set: Average loss: 0.5867, Accuracy: 80%

Epoch: 2


100%|██████████| 32/32 [00:00<00:00, 93.75it/s]


...


Train set: Average loss: 0.0000, Accuracy: 100%
Test set: Average loss: 1.4044, Accuracy: 89%

Epoch: 9


100%|██████████| 32/32 [00:00<00:00, 94.31it/s]



Train set: Average loss: 0.0000, Accuracy: 100%
Test set: Average loss: 1.4354, Accuracy: 89%

Epoch: 10


100%|██████████| 32/32 [00:00<00:00, 94.32it/s]



Train set: Average loss: 0.0000, Accuracy: 100%
Test set: Average loss: 1.4680, Accuracy: 89%
Training is ended!


In [42]:
res_3=main(model_3, optimizer_3)

test_accuracy,▁▁▂▆▇▇██▇▆
test_loss,▅█▂▂▂▁▁▁▂▂
train_accuracy,▅▃▆▃▃▁▃▂█▃
train_loss,█▆▇▃▁▅▆▃▃▅
test_accuracy,0.25781
test_loss,0.84087
train_accuracy,0.27441
train_loss,0.95314



Epoch: 1


100%|██████████| 32/32 [00:00<00:00, 84.87it/s]



Train set: Average loss: 0.3650, Accuracy: 69%
Test set: Average loss: 0.5591, Accuracy: 80%

Epoch: 2


100%|██████████| 32/32 [00:00<00:00, 91.21it/s]


...


Train set: Average loss: 0.0002, Accuracy: 100%
Test set: Average loss: 1.1957, Accuracy: 88%

Epoch: 9


100%|██████████| 32/32 [00:00<00:00, 94.67it/s]



Train set: Average loss: 0.0005, Accuracy: 100%
Test set: Average loss: 1.2411, Accuracy: 88%

Epoch: 10


100%|██████████| 32/32 [00:00<00:00, 95.28it/s]


Train set: Average loss: 0.0001, Accuracy: 100%
Test set: Average loss: 1.2581, Accuracy: 89%
Training is ended!


In [45]:
x_test[:256]

,review_full
462975,One of the best places I have been to recently...
491430,"OK, I admit this isn't everyone's cup of tea. ..."
346774,Great food. Outrageously priced. Ambience real...
74432,Great meal! Menu changes daily based on the fr...
642552,We had dinner here. There was 4 of us and the ...
...,...
150623,This is a very trendy restaurant. Excelent foo...
330683,"We were looking for a nice sit-down, table ser..."
210654,Very nice place and good food. Service was ve...
148606,First venison sausage of the season.Pretty Chr...


In [46]:
predictions_np = res_1.cpu().numpy()
df = pd.DataFrame(predictions_np, columns=['prediction'])
df['review'] = x_test.reset_index(drop=True)[:256]
df['ans'] = y_test.reset_index(drop=True)[:256]
df=df[['review', 'ans','prediction']]
df

,review,ans,prediction
0,One of the best places I have been to recently...,1,1
1,"OK, I admit this isn't everyone's cup of tea. ...",1,1
2,Great food. Outrageously priced. Ambience real...,1,1
3,Great meal! Menu changes daily based on the fr...,1,1
4,We had dinner here. There was 4 of us and the ...,0,0
...,...,...,...
251,This is a very trendy restaurant. Excelent foo...,1,1
252,"We were looking for a nice sit-down, table ser...",1,1
253,Very nice place and good food. Service was ve...,1,1
254,First venison sausage of the season.Pretty Chr...,1,1


In [47]:
df[df['ans']==df['prediction']].shape[0]/df.shape[0]

0.87890625

In [49]:
#вот и та accuracy которая была посчитана на последней эпохе у первой модели

In [48]:
#кстати качество получилось лучше чем случайным лесом на семинаре

In [71]:
#функция для прогнозов на новых данных:

In [51]:
def predict_pn(texts, model, vectorizer, device='cpu'):

    # Преобразуем текст в векторы через обученный векторизатор
    tfidf_test = vectorizer.transform(texts)
    test_tfidf_dense = scipy.sparse.csr_matrix.todense(tfidf_test)
    X_test_tensor = torch.tensor(test_tfidf_dense, dtype=torch.float32)
    test_dataset = TensorDataset(X_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=CFG.test_batch_size, shuffle=False)

    model.eval()
    all_pred = []
    with torch.no_grad():
        for (data,) in test_loader:
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            all_pred.append(pred)

    all_pred = torch.cat(all_pred, dim=0)
    all_pred = all_pred.squeeze(dim=1)
    all_pred = all_pred.cpu().numpy()

    df = pd.DataFrame({'review': texts, 'prediction': all_pred})
    return df

In [73]:
texts = ["it was great!", "awful...",  "I like this", "good", "Great service and friendly staff!","Loved the atmosphere!","Clean, fast, and reliable.",
          "Would definitely come back.", "Exceeded my expectations.", "Very disappointing experience.","Not worth the price.","Service was slow and rude.",
          "I wouldn’t recommend it.","Room was dirty and noisy.", "don't like it", "it wasn't tasty"]
preds = predict_pn(texts, model_1, vectorizer, device='cuda')
preds

,review,prediction
0,it was great!,1
1,awful...,0
2,I like this,1
3,good,1
4,Great service and friendly staff!,1
5,Loved the atmosphere!,1
6,"Clean, fast, and reliable.",1
7,Would definitely come back.,1
8,Exceeded my expectations.,1
9,Very disappointing experience.,0


In [68]:
#модель хорошо отличает положительные отзывы, но иногда путается с отрицательными из-за частицы "не".